In [1]:
import sys
import os

# Get the current working directory of the notebook
notebook_dir = os.getcwd()

# Go up one level to the project folder
project_dir = os.path.dirname(notebook_dir)

# Add the project folder to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

# Now you should be able to import from the graphmaker package
from graphmaker import config

In [2]:
import datetime
import glob
import json
import os
from concurrent.futures import ThreadPoolExecutor
from typing import List, Tuple

import requests
from tqdm import tqdm


def api_call(method: str, str_param: str = ""):
    url = f"{config.BASE_URL}/{method}/{str_param}"
    r = requests.get(url)
    json = r.json()
    if json["message"] == "OK":
        return json["data"]
    else:
        return None


def parse_projects() -> dict:
    """Получение данных о проектах"""
    response = api_call("projects?statusIds[]=2")
    return response["projects"]

def project_team(project_id: int) -> Tuple[int, dict]:
    team = api_call("project/students", str_param=project_id)
    team = team["activeMembers"]
    return project_id, team


def detailed_project_info(pid: str, return_field: str = None):
    r = requests.get(f"https://cabinet.miem.hse.ru/public-api/project/body/{pid}")
    data = r.json()["data"]
    if return_field is None:
        return pid, data
    else:
        return pid, data[return_field]


def parse_from_cabinet():
    data = parse_projects()

    pids = [i["id"] for i in data]
    teams = {}
    detailed_infos = {}

    with ThreadPoolExecutor(max_workers=12) as executor:
        for pid, team in tqdm(executor.map(project_team, pids), total=len(pids)):
            teams[pid] = team

        for pid, info in tqdm(
            executor.map(detailed_project_info, pids), total=len(pids)
        ):
            detailed_infos[pid] = info

    for p in tqdm(data):
        pid = p["id"]
        p["detailed_team"] = teams[pid]
        detailed_info = detailed_infos[pid]
        p["projectIndustryLabel"] = detailed_info.get(
            "projectIndustryLabel", "Неизвестно"
        )
        p["typeDesc"] = detailed_info.get("typeDesc", "Неизвестно")
        p["leaders"] = detailed_info.get("leaders")

        for l in p["leaders"]:
            try:
                del l["pic"]
            except KeyError:
                continue

    return data


def get_projects():
    now = datetime.datetime.now()
    day = now.day
    month = now.month
    year = now.year

    projects_filename = f"projects_{day}_{month}_{year}.json"
    projects_filename_path = f"{config.ROOT_DIR}/data/{projects_filename}"
    if os.path.exists(projects_filename_path):
        with open(projects_filename_path, "r") as f:
            return json.load(f)

    # if todays file does not exist
    try:
        data = parse_from_cabinet()
        with open(projects_filename_path, "w") as f:
            json.dump(data, f, ensure_ascii=False)
    except:
        all_projects_caches = glob.glob(f"{config.ROOT_DIR}/data/projects*")
        with open(all_projects_caches[-1], "r") as f:
            return json.load(f)

    return data


if __name__ == "__main__":
    from pprint import pprint

    data = get_projects()
    print(data[0])

100%|██████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<?, ?it/s]

{'id': 1588, 'status': 'control_passed', 'statusDesc': 'Рабочий', 'nameRus': ' Проект ИСП РАН: Разработка инструментов верификации САПР цифровых СБИС', 'head': 'Камкин Александр Сергеевич', 'directionHead': 'Романов Александр Юрьевич', 'type': 'soft', 'typeDesc': 'Неизвестно', 'typeId': 2, 'statusId': 2, 'dateCreated': '06.09.2023 10:55:31', 'vacancies': 0, 'team': ['Чижмар М.М.', 'Кашинцев И.К.', 'Ржевская М.М.', 'Портнягин Е.', 'Грицун Т.Ю.'], 'vacancyData': [], 'number': 1588, 'hoursCount': 239, 'initiatorEmail': 'akamkin@hse.ru', 'payed': True, 'projectOfficeControl': True, 'studentsCount': 5, 'searchString': 'Камкин Александр Сергеевич 1588  Проект ИСП РАН: Разработка инструментов верификации САПР цифровых СБИС Чижмар М.М. Кашинцев И.К. Ржевская М.М. Портнягин Е. Грицун Т.Ю.', 'clientLogo': '', 'detailed_team': [{'fullName': 'Грицун Тимофей Юрьевич', 'name': 'Грицун Тимофей Юрьевич', 'userId': 3366, 'id': 3366, 'email': ['tyugritsun@edu.hse.ru'], 'first_name': 'Тимофей', 'last_nam